Import bibliotekii, wczytanie plików.
Dodanie kolumny stanu.

In [ ]:
import pandas as pd
restaurants = pd.read_csv('restaurants.csv')
menu = pd.read_csv('restaurant-menus.csv')
merged = restaurants.merge(menu, how='left', left_on='id', right_on='restaurant_id' , sort=False, suffixes=('_rest', '_menu'))
restaurants['state'] = [str(a).replace(str(b), '').strip()[-3:-1] for a, b in zip(restaurants['full_address'], restaurants['zip_code'])]
restaurants.loc[restaurants['zip_code'].str.len() == 2, ['zip_code']] = '00000'
restaurants.loc[restaurants['zip_code'].isin(['75070']), ['state']] = 'TX'
restaurants.loc[restaurants['state'].isin([', ','an','U']),['state']] = 'PR'
restaurants['state'].unique()
restaurants[restaurants['state']=='']


Wyodrębnienie danych dla poszczególnych kategorii cenowych.

In [ ]:
cheap = restaurants[restaurants['price_range']=='$']
moderate = restaurants[restaurants['price_range']=='$$']
expensive = restaurants[restaurants['price_range']=='$$$']
superexpensive = restaurants[restaurants['price_range']=='$$$$']

Ile jest tanich restauracji w poszczególny stanach?

In [ ]:
cheapsy= moderate.groupby('state', as_index=False)['price_range'].count()
cheapsy.rename(columns = {'price_range': 'cheap #'}, inplace=True)
cheapsy

Ile jest restauracji o umiarkowanych cenach?

In [ ]:
moderasy = moderate.groupby('state', as_index=False)['price_range'].count()
moderasy.rename(columns = {'price_range': 'moderate #'}, inplace=True)
moderasy

Ile jest drogich restauracji?

In [ ]:
expensy = expensive.groupby('state', as_index=False)['price_range'].count()
expensy.rename(columns = {'price_range': 'expensive #'}, inplace=True)
expensy

Ile jest BARDZO drogich restauracji?

In [ ]:
superesy = superexpensive.groupby('state', as_index=False)['price_range'].count()
superesy.rename(columns = {'price_range': 'superexpensive #'}, inplace=True)

superesy

Połączenie wszystkich kategorii w tabelę całościową i wyliczenie wartości całkowitej (tj. ile jest restauracji z zakresem cen w danym stanie)

In [ ]:
chm = cheapsy.merge(moderasy, how='outer', on ='state')
exmch = chm.merge(expensy, how='outer', on ='state')
all = exmch.merge(superesy, how='outer', on ='state')
#usuniecie wartosci NaN
all = all.fillna(0)
all['total'] = all['cheap #'] + all['moderate #'] + all['expensive #'] + all['superexpensive #']
all

Ile procentowo stanowią określone kategorie cenowe w konkretnych stanach?

In [ ]:
all['cheap #'] = all['cheap #'] / all['total'] * 100
all.rename(columns = {'cheap #': 'cheap %'}, inplace=True)
all['moderate #'] = all['moderate #'] / all['total'] * 100
all.rename(columns = {'moderate #': 'moderate %'}, inplace=True)
all['expensive #'] = all['expensive #'] / all['total'] * 100
all.rename(columns = {'expensive #': 'expensive %'}, inplace=True)
all['superexpensive #'] = all['superexpensive #'] / all['total'] * 100
all.rename(columns = {'superexpensive #': 'superexpensive %'}, inplace=True)

In [57]:
#zaokrąglenie do dwóch miejsc po przecinku
all.round(2)

,state,cheap %,moderate %,expensive %,superexpensive %,total
0,,54.65,44.19,1.16,0.00,172.0
1,AL,67.76,31.61,0.63,0.00,949.0
2,DC,70.73,27.00,2.09,0.18,1100.0
3,ID,65.22,34.78,0.00,0.00,23.0
4,IL,76.63,23.37,0.00,0.00,184.0
5,MD,74.16,24.86,0.98,0.00,712.0
6,MN,58.97,41.03,0.00,0.00,39.0
7,OH,33.33,66.67,0.00,0.00,15.0
8,OR,82.34,17.23,0.42,0.00,708.0
9,PR,91.28,8.72,0.00,0.00,149.0


Co nam dają te informacje?  
Mozna załozyc, ze otworzenie drogiej restauracji w stanie, w ktorym jest znaczna przewaga restauracji tanich   
(np. VA: na prawie 8k restauracji ponad 80% stanowią tanie restauracje) nie miałoby większego sensu.  

Lepszym pomysłem byłby stan, w którym mamy do czynienia z nizszym stosunkiem restauracji o niskim  
zakresie cenowym i pojedyncze drogie restauracje: np. WI, WV, VT. 
Niestety rzuca się tu w oczy niedoskonałość naszych danych, tj. w niektórych stanach mamy tysiące restauracji, a w innych kilka/kilkadziesiąt.

